In [1]:
pip install dpyacl

     |████████████████████████████████| 48 kB 1.9 MB/s eta 0:00:011
  Created wheel for dpyacl: filename=dpyacl-0.3-py3-none-any.whl size=53903 sha256=cf1778dfc9dcf6a82f4857d00444e6a01a82a5f1ffaab847720696478dea3211
  Stored in directory: /home/jovyan/.cache/pip/wheels/84/78/bc/90f1c59739cc83e011143a4436d31b57dbfef0b140ad84355a
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13698 sha256=fb2d7a43c37884ed11a35ea6181cb0c77a0ac48a749ecd9d5d5e840226213449
  Stored in directory: /home/jovyan/.cache/pip/wheels/8c/76/0b/eb9eb3da7e2335e3577e3f96a0ae9f74f206e26457bd1a2bc8
Successfully built dpyacl prettytable
Note: you may need to restart the kernel to use updated packages.


In [26]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=2,
                n_workers=50, memory_limit='2GB')
client

/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37497 instead
  http_address["port"], self.http_server.port


Client Scheduler: inproc://10.12.19.209/93/118 Dashboard: /user/dask-dask-examples-glqyapyz/proxy/37497/status,Cluster Workers: 50 Cores: 100 Memory: 100.00 GB


In [27]:
import dask.array as da
from sklearn.datasets import make_classification, load_diabetes, load_iris

X, y = load_iris(return_X_y=True)
X = da.from_array(X)
y = da.from_array(y)


In [28]:
X

dask.array<array, shape=(150, 4), dtype=float64, chunksize=(150, 4), chunktype=numpy.ndarray>

In [29]:
from sklearn.linear_model import LogisticRegression
from dpyacl.core.stop_criteria import MaxIteration
from dpyacl.strategies.single_label import QueryMarginSampling
from dpyacl.metrics.evaluation import Accuracy, F1, HammingLoss

ml_technique = LogisticRegression(solver='sag')
stopping_criteria = MaxIteration(50)
query_strategy = QueryMarginSampling()
performance_metrics = [Accuracy(),
                       F1(average='weighted'),
                       HammingLoss()]

In [30]:
from dpyacl.experiment.context import HoldOutExperiment
from dpyacl.oracle import SimulatedOracle
from dpyacl.scenario import PoolBasedSamplingScenario

# init the ALExperiment
experiment = HoldOutExperiment(
    X,
    y,
    scenario_type=PoolBasedSamplingScenario,
    ml_technique=ml_technique,
    performance_metrics=performance_metrics,
    query_strategy=query_strategy,
    oracle=SimulatedOracle(labels=y),
    stopping_criteria=stopping_criteria,
    self_partition=True,
    test_ratio=0.3,
    initial_label_rate=0.05,
    all_class=True
)

In [31]:
%%time
result = experiment.evaluate(client= client, verbose=True)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Label: [1], Cost: [1]

| round | initially labeled data | number of queries | cost | accuracy_score: |  f1_score:   | hamming_loss: |
|   0   |    5 (4.76% of all)    |         1         |  0   |   0.600 ± 0.00  | 0.502 ± 0.00 |  0.400 ± 0.00 |

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Label: [1], Cost: [1]
|   0   |    5 (4.76% of all)    |         2         |  0   |   0.789 ± 0.19  | 0.740 ± 0.24 |  0.211 ± 0.19 |

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


...

Label: [0], Cost: [1]
|   0   |    5 (4.76% of all)    |         48        |  0   |   0.952 ± 0.07  | 0.948 ± 0.09 |  0.048 ± 0.07 |

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Label: [1], Cost: [1]
|   0   |    5 (4.76% of all)    |         49        |  0   |   0.952 ± 0.07  | 0.948 ± 0.09 |  0.048 ± 0.07 |

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Label: [2], Cost: [1]
|   0   |    5 (4.76% of all)    |         50        |  0   |   0.953 ± 0.07  | 0.949 ± 0.08 |  0.047 ± 0.07 |CPU times: user 45 s, sys: 16.1 s, total: 1min 1s
Wall time: 1min 1s


In [ ]:
from dpyacl.experiment import ExperimentAnalyserFactory

query_analyser = ExperimentAnalyserFactory.experiment_analyser(
            performance_metrics= [
                metric.metric_name
                    for metric in performance_metrics
            ],
            method_name=query_strategy.query_function_name,
            method_results=result,
            type="queries"
    )
query_analyser.plot_learning_curves(
    title='Active Learning experiment results')